# Essential Libraries

In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2, csv
import pandas as pd
import numpy as np
import random
import splitfolders
import shutil
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model

2025-06-30 12:05:26.095218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [42]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path)

100%|██████████| 750M/750M [10:46<00:00, 1.22MB/s] 

Extracting files...


Path to dataset files: /Users/muhammadabdelmohsen/.cache/kagglehub/datasets/jessicali9530/stanford-dogs-dataset/versions/2


### Cleaning & Pre-processing

In [15]:
ImagesDir = "/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification/Images"
trainDir = "/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification/train"
testDir = "/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification/test"
valDir = "/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification/val"

In [ ]:
for i in os.listdir(ImagesDir):
    if i == ".DS_Store":
        continue
    name = i.split('-', 1)[1]   # Split at first dash and take the second part
    print(name)


In [8]:
Folders = set(os.listdir(ImagesDir))
print("Unique Dog Breeds Found:", len(Folders))
Folders

Unique Dog Breeds Found: 124


{'.DS_Store',
 'Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'Brittany_spaniel',
 'Cardigan',
 'Chesapeake_Bay_retriever',
 'Chihuahua',
 'Dandie_Dinmont',
 'Doberman',
 'English_foxhound',
 'English_setter',
 'English_springer',
 'EntleBucher',
 'Eskimo_dog',
 'French_bulldog',
 'German_shepherd',
 'German_short-haired_pointer',
 'Gordon_setter',
 'Great_Dane',
 'Great_Pyrenees',
 'Greater_Swiss_Mountain_dog',
 'Ibizan_hound',
 'Irish_setter',
 'Irish_terrier',
 'Irish_water_spaniel',
 'Irish_wolfhound',
 'Italian_greyhound',
 'Japanese_spaniel',
 'Kerry_blue_terrier',
 'Labrador_retriever',
 'Lakeland_terrier',
 'Leonberg',
 'Lhasa',
 'Maltese_dog',
 'Mexican_hairless',
 'Newfoundland',
 'Norfolk_terrier',
 'Norwegian_elkhound',
 'Norwic

In [ ]:
# add all the folders in the foldersSet to the trainDir, testDir, valDir
if not os.path.exists(trainDir):
    os.makedirs(trainDir)
if not os.path.exists(testDir):
    os.makedirs(testDir)
if not os.path.exists(valDir):
    os.makedirs(valDir)
# Create subdirectories for train, test, and validation sets dog breeds
for folder in Folders:

    if folder == ".DS_Store":
        continue
    folderPath = os.path.join(ImagesDir, folder)
    if not os.path.exists(os.path.join(trainDir, folder)):
        os.makedirs(os.path.join(trainDir, folder))
    if not os.path.exists(os.path.join(testDir, folder)):
        os.makedirs(os.path.join(testDir, folder))
    if not os.path.exists(os.path.join(valDir, folder)):
        os.makedirs(os.path.join(valDir, folder))


In [17]:
random.seed(42)

for breed in os.listdir(ImagesDir):
    breed_path = os.path.join(ImagesDir, breed)
    if breed == ".DS_Store":
        continue

    images = [f for f in os.listdir(breed_path) if os.path.isfile(os.path.join(breed_path, f))]
    random.shuffle(images)

    nTotal = len(images)
    nTrain = int(0.8 * nTotal)
    nVal = int(0.1 * nTotal)
    nTest = nTotal - nTrain - nVal

    train_files = images[:nTrain]
    val_files = images[nTrain:nTrain + nVal]
    test_files = images[nTrain + nVal:]

    for split_name, split_files in zip(
        ["train", "val", "test"],
        [train_files, val_files, test_files]
    ):
        split_breed_dir = os.path.join("/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/DogBreedsClassification", split_name, breed)
        os.makedirs(split_breed_dir, exist_ok=True)

        for file in split_files:
            src = os.path.join(breed_path, file)
            dst = os.path.join(split_breed_dir, file)
            shutil.copy2(src, dst)

    print(f"Breed: {breed} -> Train: {nTrain}, Val: {nVal}, Test: {nTest}")

print("✅ Done splitting!")

Breed: Australian_terrier -> Train: 156, Val: 19, Test: 21
Breed: toy_poodle -> Train: 120, Val: 15, Test: 16
Breed: Great_Pyrenees -> Train: 170, Val: 21, Test: 22
Breed: Maltese_dog -> Train: 201, Val: 25, Test: 26
Breed: Norwich_terrier -> Train: 148, Val: 18, Test: 19
Breed: whippet -> Train: 149, Val: 18, Test: 20
Breed: Boston_bull -> Train: 145, Val: 18, Test: 19
Breed: Irish_setter -> Train: 124, Val: 15, Test: 16
Breed: Rottweiler -> Train: 121, Val: 15, Test: 16
Breed: kelpie -> Train: 122, Val: 15, Test: 16
Breed: schipperke -> Train: 123, Val: 15, Test: 16
Breed: Leonberg -> Train: 168, Val: 21, Test: 21
Breed: Welsh_springer_spaniel -> Train: 120, Val: 15, Test: 15
Breed: Pomeranian -> Train: 175, Val: 21, Test: 23
Breed: Pekinese -> Train: 119, Val: 14, Test: 16
Breed: Irish_wolfhound -> Train: 174, Val: 21, Test: 23
Breed: Blenheim_spaniel -> Train: 150, Val: 18, Test: 20
Breed: basenji -> Train: 167, Val: 20, Test: 22
Breed: African_hunting_dog -> Train: 135, Val: 16, T

# Data Augmentation

In [38]:
trainData = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
testData = ImageDataGenerator(rescale=1./255)
valData = ImageDataGenerator(rescale=1./255)

In [39]:
trainData = trainData.flow_from_directory(
    trainDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

testData = testData.flow_from_directory(
    testDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

valData = valData.flow_from_directory(
    valDir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 16418 images belonging to 120 classes.
Found 2153 images belonging to 120 classes.
Found 2009 images belonging to 120 classes.


In [1]:
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer = regularizers.l2(0.01)),
    MaxPooling2D((2,2), padding='valid'),
    BatchNormalization(),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling2D((2,2), padding='valid'),
    BatchNormalization(),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),
    MaxPooling2D((2,2), padding='valid'),
    BatchNormalization(),
    Dropout(0.2),
    
    Flatten(),

    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

])

NameError: name 'Sequential' is not defined

In [42]:
earlyStopping = EarlyStopping(patience=5, restore_best_weights=True, monitor=['val_loss'])
model.compile(
    optimizer=Adam(learning_rate=0.0008),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [43]:
history = model.fit(
    trainData,
    validation_data=valData,
    epochs=2,
    callbacks=[earlyStopping],
    verbose=1
)

/Users/muhammadabdelmohsen/Desktop/ML & DL & LLM Projects/ML_NN_Env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 120), output.shape=(None, 128)